### RAG stacks + ollama + Store in Vector Db

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama

# Example document
document_text = """
The Eiffel Tower was built in 1889 in Paris. It stands 324 meters tall and is one of the most visited monuments in the world.
It was originally constructed as a temporary exhibit for the 1889 World's Fair.
"""

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunks = splitter.split_text(document_text)

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Store in FAISS
vector_db = FAISS.from_texts(chunks, embedding=embeddings)

# Query example
query = "Tell me where is the Eiffel tower?"
similar_docs = vector_db.similarity_search(query, k=2)
context_text = "\n".join([doc.page_content for doc in similar_docs])

# Use LLaMA 3 via Ollama
llm = Ollama(model="llama3.1")

final_prompt = f"""
Answer the question using the context below. If the answer is not found, say "I don't know."

Context:
{context_text}

Question: {query}
Answer:
"""

response = llm(final_prompt)
print(response)

Paris.


In [ ]:
# to inspect list in the stored chunks

ids = list(vector_db.docstore._dict.keys())

for doc_id in ids:
    